In [2]:
import datetime
import tweepy
from API_keys_Brenden import api_key, api_key_secret, access_token, access_token_secret
import time
from dateutil import tz


from_zone = tz.gettz('UTC')
to_zone = tz.gettz('America/New_York')

In [8]:
# Authenticate the Tweepy API
auth = tweepy.OAuthHandler(api_key,api_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True ,wait_on_rate_limit_notify=True)

In [27]:
userID = "realDonaldTrump"


In [28]:
pages=[]
for page in tweepy.Cursor(api.user_timeline, screen_name=userID,tweet_mode='extended',include_rts=0,wait_on_rate_limit=True, wait_on_rate_limit_notify=True).pages(80):
    pages.extend(page)


In [64]:
l=[]
for tweet in pages:
    #uncomment to remove quote statuses
    #if tweet.is_quote_status:
        #continue
    text = tweet.full_text.replace('\n','') #remove new lines
    l.append([tweet.created_at, text])

In [65]:
with open('trump_tweets.txt','w',encoding='utf-8') as outfile:
    outfile.write('timestamp\ttweet\n')
    for tweet in l:
        outfile.write(str(tweet[0])+'\t'+tweet[1]+'\n')

# AP state call times

In [4]:
userID = "AP_Politics"
pages=[]
for page in tweepy.Cursor(api.user_timeline, screen_name=userID,tweet_mode='extended',include_rts=0,wait_on_rate_limit=True, wait_on_rate_limit_notify=True).pages(100):
    pages.extend(page)


In [83]:
ap=[]
for tweet in pages:
    #uncomment to remove quote statuses
    #if tweet.is_quote_status:
        #continue
    text = tweet.full_text.replace('\n','') #remove new lines
    timestamp=tweet.created_at
    timestamp=timestamp.replace(tzinfo=from_zone)
    timestamp=timestamp.astimezone(to_zone)

    ap.append([timestamp, text])

In [84]:
state_calls=[]
for tweet in ap:
    text=tweet[1].lower()
    if '#apracecall' in text and any(cand in text for cand in ['biden','trump']):
        state_calls.append(tweet)
        
state_calls=state_calls[:51]

In [85]:
e_votes={'Alabama':9,'Alaska':3,'Arizona':11,'Arkansas':6,'California':55,
        'Colorado':9,'Connecticut':7,'Delaware':3,'the District of Columbia':3,
        'Florida':29,'Georgia':16,'Hawaii':4,'Idaho':4,'Illinois':20,
        'Indiana':11,'Iowa':6,'Kansas':6,'Kentucky':8,'Louisiana':8,
        'Maine':4,'Maryland':10,'Massachusetts':11,'Michigan':16,
        'Minnesota': 10, 'Mississippi':6,'Missouri':10,'Montana':3,
        'Nebraska':5,'Nevada':6,'New Hampshire':4,'New Jersey': 14,
        'New Mexico': 5,'New York':29,'North Carolina':15,'North Dakota':3,
        'Ohio': 18,'Oklahoma':7,'Oregon': 7,'Pennsylvania':20,
        'Rhode Island':4,'South Carolina':9,'South Dakota':3,
        'Tennessee':11,'Texas':38,'Utah':6,'Vermont':3,'Virginia':13,
        'Washington':12,'West Virginia': 5,'Wisconsin':10,'Wyoming':3}
         

In [99]:
state_list=[]
time_list=[]
votes_list=[]
winner_list=[]
for i in range(51):
    winner=state_calls[i][1].split('wins')[0].split()[-1].strip()
    state = state_calls[i][1].split('wins')[1].split('.')[0].strip()
    votes = e_votes[state]
    time = state_calls[i][1].split('at')[1].split('#')[0]
    hour_min = time.split()[0]
    am_pm = time.split()[1]

    timezone = time.split('m.')[1].split('.')[0].strip()
    year=2020
    month=11
    day=state_calls[i][0].day
    hour=int(hour_min.split(':')[0])
    mins=int(hour_min.split(':')[1])
    
    if am_pm=='p.m.':
        hour+=12

    if timezone=='EST':
        pass
    elif timezone=='CST':
        hour+=1
    elif timezone=='MST':
        hour+=2
    elif timezone=='PST':
        hour+=3
    elif timezone=='Alaska Standard Time':
        hour+=4
    elif timezone=='HST':
        hour+=5
    else:
        print('uh oh')
        
    if hour>23:
        hour=hour-24
        day+=1
    
    call_time = datetime.datetime(year,11,day,hour,mins,0, tzinfo=tz.gettz('America/New_York'))
    
    state_list.append(state)
    votes_list.append(votes)
    time_list.append(call_time)
    winner_list.append(winner)
    
  

In [100]:
import pandas as pd
df=pd.DataFrame()

In [101]:
df['state']=state_list
df['call_time']=time_list
df['votes']=votes_list
df['winner']=winner_list

In [102]:
df.to_csv('AP_call_times.csv',index=False)

# get all of trump tweets including rts

In [34]:
#Get trump tweet numbers by day
userID = "realDonaldTrump"
pages=[]
for page in tweepy.Cursor(api.user_timeline, screen_name=userID,include_rts=0).pages(90):
    pages.extend(page)
    
len(pages)

1093

In [44]:
l=[]
for tweet in pages:
    #uncomment to remove quote statuses
    #if tweet.is_quote_status:
        #continue
    text = tweet.text.replace('\n','') #remove new lines
    l.append([tweet.created_at, text])
    
with open('trump_tweets_all.txt','w',encoding='utf-8') as outfile:
    outfile.write('timestamp\ttweet\n')
    for tweet in l:
        outfile.write(str(tweet[0])+'\t'+tweet[1]+'\n')